In [1]:
from MultiEurlexDataset import MultiEurlexDataset
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline, EarlyStoppingCallback
from torch.utils.data import RandomSampler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import torch.nn as nn

In [2]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
device='cuda'
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base', padding=True, trunaction=True)
training_dataset = MultiEurlexDataset(languages = ['hr'], tokenizer=tokenizer)
eval_dataset = MultiEurlexDataset(split='validation',languages = ['hr'], tokenizer=tokenizer)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Reusing dataset multi_eurlex (C:\Users\LPavl\.cache\huggingface\datasets\multi_eurlex\all_languages\1.0.0\8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


Reusing dataset multi_eurlex (C:\Users\LPavl\.cache\huggingface\datasets\multi_eurlex\all_languages\1.0.0\8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


In [4]:
collator_fn = DataCollatorWithPadding(tokenizer,return_tensors="pt", padding="max_length")

In [5]:
metric = f1_score

def compute_metrics(eval_pred):
    label_pred, label_true = eval_pred
    sigmoid = nn.Sigmoid()
    label_pred=sigmoid(torch.tensor(label_pred)).numpy()
    label_pred[label_pred < 0.5] = 0
    label_pred[label_pred >= 0.5] = 1
    result={}
    result['f1_score'] = metric(label_true,label_pred, average='micro')
    return result

In [6]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=1e-3,
    save_strategy='epoch',
    eval_steps=50,
    evaluation_strategy='steps',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
)

In [7]:
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=21, problem_type="multi_label_classification")


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=collator_fn,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=5),
    ]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, function_to_apply='sigmoid')

In [10]:
os.environ["WANDB_DISABLED"] = "true"
#trainer.train()

In [11]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2500
  Batch size = 4


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'eval_loss': 0.7108078631713987,
 'eval_f1_score': 0.3145660103767814,
 'eval_runtime': 63.3227,
 'eval_samples_per_second': 39.48,
 'eval_steps_per_second': 9.87}

In [12]:
model(input_ids=training_dataset[0]['input_ids'].unsqueeze(0).to(device),attention_mask=training_dataset[0]['attention_mask'].unsqueeze(0).to(device))

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2723, -0.0026,  0.1791,  0.0111, -0.4315, -0.1163,  0.3376, -0.2125,
          0.2792,  0.0513,  0.1881, -0.0364,  0.0949,  0.1153, -0.1765,  0.2051,
         -0.0643,  0.1774,  0.3711,  0.0736,  0.0186]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)